In [92]:
import pandas as pd
import numpy as np
import scipy
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing  import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score

In [35]:
games2000 = pd.read_csv("final/gamesFinal2000.csv")
games2001 = pd.read_csv("final/gamesFinal2001.csv")
games2002 = pd.read_csv("final/gamesFinal2002.csv")
games2003 = pd.read_csv("final/gamesFinal2002.csv")
games2004 = pd.read_csv("final/gamesFinal2004.csv")
games2005 = pd.read_csv("final/gamesFinal2005.csv")

In [41]:
def changeMinutesToSeconds(row):
    away = row["possession_time_away_team"].split(":")
    away_seconds = (int(away[0])*60)+int(away[1])
    
    home = row["possession_time_home_team"].split(":")
    home_seconds = (int(home[0])*60)+int(home[1])
    
    row["possession_time_away_team"] = away_seconds
    row["possession_time_home_team"] = home_seconds
    return row

games2000_time =  games2000.apply(changeMinutesToSeconds, axis=1)
games2001_time =  games2001.apply(changeMinutesToSeconds, axis=1)
games2002_time =  games2002.apply(changeMinutesToSeconds, axis=1)
games2003_time =  games2003.apply(changeMinutesToSeconds, axis=1)
games2004_time =  games2004.apply(changeMinutesToSeconds, axis=1)
games2005_time =  games2005.apply(changeMinutesToSeconds, axis=1)

In [42]:

games2000_clean = games2000_time.drop(["market_home_team","market_away_team","index_away_team","index_home_team","Unnamed: 0","Unnamed: 0.1_away_team","Unnamed: 0.1_home_team", "Unnamed: 0_away_team","Unnamed: 0_home_team", "id_home_team", "id_away_team",
                                 "sr_id.1_home_team","reference.1_home_team","sr_id.1_away_team","reference.1_away_team", "sr_id_away_team", "sr_id_home_team",
                                 ], axis=1)
games2001_clean = games2001_time.drop(["market_home_team","market_away_team","index_away_team","index_home_team","Unnamed: 0","Unnamed: 0.1_away_team","Unnamed: 0.1_home_team", "Unnamed: 0_away_team","Unnamed: 0_home_team", "id_home_team", "id_away_team",
                                 "sr_id.1_home_team","reference.1_home_team","sr_id.1_away_team","reference.1_away_team", "sr_id_away_team", "sr_id_home_team",
                                 ], axis=1)
games2002_clean = games2002_time.drop(["market_home_team","market_away_team","index_away_team","index_home_team","Unnamed: 0","Unnamed: 0.1_away_team","Unnamed: 0.1_home_team", "Unnamed: 0_away_team","Unnamed: 0_home_team", "id_home_team", "id_away_team",
                                 "sr_id.1_home_team","reference.1_home_team","sr_id.1_away_team","reference.1_away_team", "sr_id_away_team", "sr_id_home_team",
                                 ], axis=1)
games2003_clean = games2003_time.drop(["market_home_team","market_away_team","index_away_team","index_home_team","Unnamed: 0","Unnamed: 0.1_away_team","Unnamed: 0.1_home_team", "Unnamed: 0_away_team","Unnamed: 0_home_team", "id_home_team", "id_away_team",
                                 "sr_id.1_home_team","reference.1_home_team","sr_id.1_away_team","reference.1_away_team", "sr_id_away_team", "sr_id_home_team",
                                 ], axis=1)
games2004_clean = games2004_time.drop(["market_home_team","market_away_team","index_away_team","index_home_team","Unnamed: 0","Unnamed: 0.1_away_team","Unnamed: 0.1_home_team", "Unnamed: 0_away_team","Unnamed: 0_home_team", "id_home_team", "id_away_team",
                                 "sr_id.1_home_team","reference.1_home_team","sr_id.1_away_team","reference.1_away_team", "sr_id_away_team", "sr_id_home_team",
                                 ], axis=1)
games2005_clean = games2005_time.drop(["market_home_team","market_away_team","index_away_team","index_home_team","Unnamed: 0","Unnamed: 0.1_away_team","Unnamed: 0.1_home_team", "Unnamed: 0_away_team","Unnamed: 0_home_team", "id_home_team", "id_away_team",
                                 "sr_id.1_home_team","reference.1_home_team","sr_id.1_away_team","reference.1_away_team", "sr_id_away_team", "sr_id_home_team",
                                 ], axis=1)

In [50]:
games2000_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 37 columns):
alias_away_team                 248 non-null object
alias_home_team                 248 non-null object
avg_gain_away_team              248 non-null float64
avg_gain_home_team              248 non-null float64
fumbles_away_team               248 non-null float64
fumbles_home_team               248 non-null float64
lost_fumbles_away_team          248 non-null float64
lost_fumbles_home_team          248 non-null float64
name_away_team                  248 non-null object
name_home_team                  248 non-null object
penalties_away_team             248 non-null float64
penalties_home_team             248 non-null float64
penalty_yards_away_team         248 non-null float64
penalty_yards_home_team         248 non-null float64
play_count_away_team            248 non-null float64
play_count_home_team            248 non-null float64
points_away_team                248 non-null int64


In [122]:
games_comp =  pd.concat([games2000_clean, games2001_clean, games2002_clean, games2003_clean, games2004_clean], axis=0, sort=False)
games_comp

,alias_away_team,alias_home_team,avg_gain_away_team,avg_gain_home_team,fumbles_away_team,fumbles_home_team,lost_fumbles_away_team,lost_fumbles_home_team,name_away_team,name_home_team,...,rush_plays_home_team,safeties_away_team,safeties_home_team,total_yards_away_team,total_yards_home_team,turnovers_away_team,turnovers_home_team,used_timeouts_away_team,used_timeouts_home_team,winner
0,DET,NO,3.2,3.9,2.0,5.0,1.0,2.0,Lions,Saints,...,25.0,0.0,0.0,187.0,252.0,2.0,3.0,1,3,away
1,NO,ATL,5.7,6.5,0.0,0.0,0.0,0.0,Saints,Falcons,...,17.0,0.0,0.0,402.0,278.0,1.0,0.0,2,3,away
2,SF,CAR,6.9,5.4,3.0,0.0,1.0,0.0,49ers,Panthers,...,25.0,0.0,0.0,411.0,390.0,2.0,0.0,3,1,home
3,CLE,PIT,2.4,3.4,0.0,1.0,0.0,0.0,Browns,Steelers,...,45.0,0.0,0.0,104.0,248.0,3.0,0.0,1,1,home
4,CHI,PHI,3.9,4.7,1.0,2.0,0.0,1.0,Bears,Eagles,...,27.0,0.0,0.0,274.0,298.0,1.0,2.0,3,0,home
5,DEN,CIN,6.3,7.9,3.0,1.0,2.0,1.0,Broncos,Bengals,...,37.0,0.0,0.0,500.0,421.0,3.0,1.0,3,1,home
6,TEN,BAL,4.1,4.6,2.0,7.0,1.0,0.0,Titans,Ravens,...,29.0,0.0,0.0,191.0,368.0,2.0,4.0,2,2,away
7,SEA,OAK,4.4,7.6,3.0,1.0,2.0,0.0,Seahawks,Raiders,...,27.0,0.0,0.0,283.0,380.0,3.0,1.0,0,1,home
8,WAS,JAC,7.4,5.2,1.0,3.0,1.0,2.0,Redskins,Jaguars,...,26.0,0.0,0.0,399.0,385.0,2.0,4.0,2,3,away
9,NE,IND,5.1,9.7,3.0,1.0,1.0,1.0,Patriots,Colts,...,21.0,0.0,0.0,386.0,398.0,1.0,1.0,2,2,home


In [135]:
X = games_comp.drop(["name_away_team","name_home_team","points_home_team","points_away_team","winner"], axis=1)
X = pd.get_dummies(X, columns=['alias_away_team','alias_home_team'], drop_first=True)
y = games_comp[["points_home_team","points_away_team"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [137]:
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

def addWinner(row):
    if row["points_home_team"] > row["points_away_team"]:
        row["winner"] = "home"
    elif row["points_home_team"] < row["points_away_team"]:
        row["winner"] = "away"
    else:
        row["winner"] = "tie"
    return row
x = pd.DataFrame(data= y_pred, columns =["points_home_team","points_away_team"])
x =  x.apply(addWinner, axis=1)

compare = pd.DataFrame(data= games2000_clean["winner"], columns=["winner"])
compare =compare.rename(columns={"winner": "winner_true"})
result = pd.concat([compare, x], axis=1, sort=False)
correct = 0
for i, row in result.iterrows():
    if row["winner"] == row["winner_true"]:
        correct = correct+1
        
print("correct matches: %d" % correct)

correct matches: 124
